## Assignment 4 (S&DS 266)

Jeffrey Gong

### Part 1: CNN

In this notebook, I implement a CNN to predict handwritten digits from the MNIST dataset. The choice of using two convolutional layers, each followed by max pooling, was aimed at extracting and downsampling features efficiently, reducing the dimensionality while retaining important spatial features. Incorporating dropout before the fully connected layers serves as a regularization technique to prevent overfitting by randomly ignoring a subset of neurons during training, thereby increasing generalization to unseen data. The use of ReLU activation functions introduces non-linearity, allowing the network to learn complex patterns. Lastly, the network concludes with a softmax output layer, providing a probabilistic interpretation of class predictions, essential for classification tasks This also allowed me to use the Cross Entropy loss. 100 epochs as suggested in the assignment was way too time intensive, and I found that just using 10 epochs did the job just fine.  

In [2]:
import pandas as pd
import numpy as np

In [13]:
mnist_train_file = "../mnist_data/mnist_train.csv"
df_train = pd.read_csv(mnist_train_file)
mnist_test_file = "../mnist_data/mnist_test.csv"
df_test = pd.read_csv(mnist_test_file)

In [11]:
df_train.head(5)["14x14"]

0    241
1      0
2      0
3    104
4     36
Name: 14x14, dtype: int64

In [12]:
df_train.head(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_test.head(5)["14x14"]

0      0
1    233
2     57
3      0
4      0
Name: 14x14, dtype: int64

In [15]:
df_test.head(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_train.iloc[:, 1:] = df_train.iloc[:, 1:].astype('float32') / 255
df_test.iloc[:, 1:] = df_test.iloc[:, 1:].astype('float32') / 255

In [17]:
df_train.head(5)["14x14"]

0    0.945098
1    0.000000
2    0.000000
3    0.407843
4    0.141176
Name: 14x14, dtype: float32

In [18]:
df_train.head(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_test.head(5)["14x14"]

0    0.000000
1    0.913725
2    0.223529
3    0.000000
4    0.000000
Name: 14x14, dtype: float32

In [20]:
df_test.head(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First convolutional layer that takes 1 input channel (2d image) 
        # and outputs 32 convolutional features, with a square kernel size of 3
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # Second convolutional layer that takes 32 input layers
        # and outputs 64 features with a square kernel size of 3
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Dropout to prevent overfitting
        self.dropout = nn.Dropout(0.25)
        # Fully connected layer taking the flattened output from conv2 
        # and outputting 10 classes
        self.fc1 = nn.Linear(64*7*7, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        # Apply the first convolution, followed by a ReLU non-linearity and a max pooling layer
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # Apply the second convolution, followed by a ReLU non-linearity and a max pooling layer
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        # Apply dropout
        x = self.dropout(x)
        # Flatten the tensor to pass into the fully connected layer
        x = torch.flatten(x, 1)
        # Apply the first fully connected layer and ReLU non-linearity
        x = F.relu(self.fc1(x))
        # Apply the second fully connected layer 
        x = self.fc2(x)
        # return the log softmax of x along dimension 1, which is necessary for for cross entropy loss
        output = F.log_softmax(x, dim=1)
        return output
        

In [23]:
X_train = df_train.drop('label', axis=1).values
Y_train = df_train['label'].values

X_test = df_test.drop('label', axis=1).values
Y_test = df_test['label'].values

In [25]:
X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

In [26]:
tensor_X_train = torch.Tensor(X_train)
tensor_Y_train = torch.Tensor(Y_train).long()

tensor_dataset = torch.utils.data.TensorDataset(tensor_X_train, tensor_Y_train)

In [27]:
train_loader = torch.utils.data.DataLoader(tensor_dataset, batch_size=64, shuffle=True)

In [31]:
# Training loop
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 10
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}')
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    # Print average loss and accuracy every 100 epochs
    if (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, '
              f'Accuracy: {100 * correct / total:.2f}%')

Epoch 1
Epoch 2
Epoch [2/10], Loss: 0.0542, Accuracy: 98.34%
Epoch 3
Epoch 4
Epoch [4/10], Loss: 0.0306, Accuracy: 98.98%
Epoch 5
Epoch 6
Epoch [6/10], Loss: 0.0210, Accuracy: 99.31%
Epoch 7
Epoch 8
Epoch [8/10], Loss: 0.0161, Accuracy: 99.47%
Epoch 9
Epoch 10
Epoch [10/10], Loss: 0.0114, Accuracy: 99.61%


In [32]:
tensor_X_test = torch.Tensor(X_test)
tensor_Y_test = torch.Tensor(Y_test).long()

test_dataset = torch.utils.data.TensorDataset(tensor_X_test, tensor_Y_test)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate the accuracy of predictions
accuracy = 100 * correct / total

# Print the model accuracy on the test set
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

Accuracy of the model on the test images: 99.32%


### Results

I found that after running my model for 10 epochs, I was able to achieve a 99.32% accuracy on the test images.